In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import opacus
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus.utils.uniform_sampler import UniformWithReplacementSampler

torch.manual_seed(472368)

In [2]:
class SampleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x
    
    @staticmethod
    def train_step(model, batch):
        x, y = batch
        # x, y = x.type(torch.LongTensor), y.type(torch.LongTensor)
        out = model(x)
        loss = model.loss(out, y)
        return loss

    @staticmethod
    def test_step(model, batch):
        x, y = batch
        # x, y = x.type(torch.LongTensor), y.type(torch.LongTensor)
        out = model(x)
        loss = model.loss(out, y)
        pred = out.argmax(dim=1, keepdim=True)#.type(torch.LongTensor)
        corrects = pred.eq(y.view_as(pred)).sum().item()
        #preds = out > 0 # Predict y = 1 if P(y = 1) > 0.5
        #corrects = torch.tensor(torch.sum(preds == y).item())
        return loss, corrects

In [3]:
def train(model, train_loader, opt_func, learning_rate, num_epochs, noise_multiplier, clip_bound, delta, verbose=False, random_seed=474237):
    optimizer = opt_func(model.parameters(), learning_rate)
    privacy_engine = opacus.PrivacyEngine(
        accountant="rdp", # Use RDP-based accounting
        secure_mode=False, # Should be set to True for production use
    )
    rng = torch.Generator()
    rng.manual_seed(int(random_seed))
    model_type = type(model)
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=noise_multiplier,
        max_grad_norm=clip_bound,
        noise_generator=rng,
        loss_reduction="mean"
    )

    for epoch in range(num_epochs):
        losses = []
        for batch in train_loader:
            loss = model_type.train_step(model, batch)
            loss.backward()
            losses.append(loss)
            optimizer.step()
            optimizer.zero_grad()
        
        if verbose:
            print("Epoch {}, loss = {}".format(epoch + 1, torch.sum(loss) / len(train_loader)))

    # epsilon, alpha = optimizer.privacy_engine.get_privacy_spent(delta)
    epsilon = privacy_engine.get_epsilon(delta)
    return epsilon

def test(model, test_loader):
    with torch.no_grad():
        losses = []
        accuracies = []
        total_size = 0
        for batch in test_loader:
            total_size += len(batch[1])
            loss, corrects = model.test_step(model, batch)
            losses.append(loss)
            accuracies.append(corrects)

        average_loss = np.array(loss).sum() / total_size
        total_accuracy = np.array(accuracies).sum() / total_size
        return average_loss, total_accuracy

In [4]:
def get_mnist_dataset(train_size=30000):
    # Precomputed characteristics of the MNIST dataset
    MNIST_MEAN = 0.1307
    MNIST_STD = 0.3081
    
    train_full_dataset = datasets.MNIST(
        "mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
            ]
        ),
    )
    train_subsample_loader = DataLoader(
        train_full_dataset, train_size,
        generator=torch.Generator().manual_seed(123456789)
    )
    train_subsample = train_subsample_loader.__iter__().__next__()
    train_dataset = TensorDataset(train_subsample[0], train_subsample[1])

    test_dataset = datasets.MNIST(
        "mnist",
        train=False,
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
            ]
        ),
    )
    return train_dataset, test_dataset

def create_data_loaders(train_tensor, test_tensor, sample_rate, random_seed=4732842):
    train_loader = DataLoader(
        train_tensor,
        batch_size=int(len(train_tensor) * sample_rate),
        generator=torch.Generator().manual_seed(random_seed)
    )
    test_loader = DataLoader(test_tensor, 64)
    return train_loader, test_loader

In [5]:
def pipeline(lr, s, C, sr, ne, train_tensor=None, test_tensor=None, verbose=False):

    """
    Function that executes the whole pipeline from data download to training and running the model.
    
    Arguments
    ---------
    lr: learning rate
    C: clipping bound
    s: noise multiplier
    sr: sampling rate
    ne: number of epochs
    data: the data, if not provided then function downloads the dataset and converts it to torch tensors.
    verbose: verbosity parameter
    
    Returns
    -------
    eps: epsilon
    delta: delta
    avg_loss: average loss
    acc_total: total accuracy
    """
    
    if train_tensor is None:
        train_tensor, test_tensor = get_mnist_dataset() # Note that this function is modified to provide the adult dataset
    
    delta = 1e-5

    input_dim = train_tensor[0][0].size(dim=0)
    train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sr) # recreate loaders with different sampling rate

    model = SampleConvNet()
    epsilon = train(
        model, train_loader, torch.optim.SGD, lr, ne,
        s, C, delta, verbose=verbose
    )
    
    average_loss, total_accuracy = test(model, test_loader)
    print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))
    
    return epsilon, delta, average_loss, total_accuracy

### i)

In [6]:
# Data preparation
train_tensor, test_tensor = get_mnist_dataset()

# Hyperparameters
delta = 1e-5

# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 1.5
# Batch size as a fraction of full data size
sample_rate = 0.004
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

In [7]:
for batch in train_loader:
    for x in batch:
        print(x.size())
        break
    break

torch.Size([120, 1, 28, 28])


In [6]:
# Data preparation
train_tensor, test_tensor = get_mnist_dataset()

# Hyperparameters
delta = 1e-5

# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 1.5
# Batch size as a fraction of full data size
sample_rate = 0.004
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

C:\Users\Administrator\anaconda3\envs\torch\lib\site-packages\opacus\privacy_engine.py:133: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
C:\Users\Administrator\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch 1, loss = 0.002683916362002492
Epoch 2, loss = 0.0017444395925849676
Epoch 3, loss = 0.003123308066278696
Epoch 4, loss = 0.0035235031973570585
Epoch 5, loss = 0.005509137641638517
Epsilon: 0.6188836247099821, Delta: 1e-05
Loss: 2.761020790785551e-07, Accuracy: 0.8917


### Accuracy on test: 89.17\% with $\epsilon = 0.619$

### ii) 

For this task, I followed a similar approach to before with the staircase optimization method keeping some parameters fixed and fine-tuning the rest, but the tests were done in an unorganized manner, trying to figure out each time towards what direction to move in the parameter space. I report two solutions I found, one focusing on accuracy and the other on $\epsilon$. If I had to choose one I would choose the solution focusing on lower $\epsilon$.

### Use full dataset

In [7]:
# Data preparation
train_tensor, test_tensor = get_mnist_dataset(train_size=60000)

### Solution focusing more on $\epsilon$

In [8]:
# Learning rate for training
learning_rate = 0.3
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.4
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 3

train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0011947514722123742
Epoch 2, loss = 0.0009345292346552014
Epoch 3, loss = 0.0014193110400810838
Epsilon: 0.34999697730814633, Delta: 1e-05
Loss: 2.0841059740632773e-07, Accuracy: 0.9095


### Solution focusing more on accuracy:

In [9]:
# Learning rate for training
learning_rate = 0.1
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 20

train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.001240633544512093
Epoch 2, loss = 0.0009662366937845945
Epoch 3, loss = 0.0007573306211270392
Epoch 4, loss = 0.0007817731820978224
Epoch 5, loss = 0.0013070753775537014
Epoch 6, loss = 0.0009868830675259233
Epoch 7, loss = 0.0006644311361014843
Epoch 8, loss = 0.0005835895426571369
Epoch 9, loss = 0.0004939452628605068
Epoch 10, loss = 0.0001406442024745047
Epoch 11, loss = 0.0005175182595849037
Epoch 12, loss = 0.00027791669708676636
Epoch 13, loss = 0.00043873387039639056
Epoch 14, loss = 0.00025171658489853144
Epoch 15, loss = 0.00016195156786125153
Epoch 16, loss = 0.0013775202678516507
Epoch 17, loss = 0.0013931195717304945
Epoch 18, loss = 0.0005723065696656704
Epoch 19, loss = 0.0006628859555348754
Epoch 20, loss = 0.00030533759854733944
Epsilon: 1.0649553579562177, Delta: 1e-05
Loss: 6.340327217913e-10, Accuracy: 0.9506


### Below are the various runs I did to end up with the above solutions. Feel free to take a look but it's quite messy.

In [21]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 0.5
# Clipping norm
clip_bound = 1.5
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.004592562559992075
Epoch 2, loss = 0.004874060861766338
Epoch 3, loss = 0.002444712445139885
Epoch 4, loss = 0.0037888793740421534
Epoch 5, loss = 0.002816338324919343
Epoch 6, loss = 0.004204682074487209
Epoch 7, loss = 0.0016670236364006996
Epoch 8, loss = 0.0017426135018467903
Epoch 9, loss = 0.00267465109936893
Epoch 10, loss = 0.003986026626080275
Epsilon: 14.148936019106882, Delta: 1e-05
Loss: 4.6494631096720696e-07, Accuracy: 0.9482


In [24]:
# Learning rate for training
learning_rate = 0.5
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 0.5
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.006581319961696863
Epoch 2, loss = 0.004990887828171253
Epoch 3, loss = 0.0025741623248904943
Epoch 4, loss = 0.003746436443179846
Epoch 5, loss = 0.002718033967539668
Epoch 6, loss = 0.00485917879268527
Epoch 7, loss = 0.002202379982918501
Epoch 8, loss = 0.0020739282481372356
Epoch 9, loss = 0.003464684821665287
Epoch 10, loss = 0.0034636142663657665
Epsilon: 14.148936019106882, Delta: 1e-05
Loss: 2.892867451009806e-09, Accuracy: 0.9516


In [25]:
# Learning rate for training
learning_rate = 0.5
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 5.
# Clipping norm
clip_bound = 1.5
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.02185060642659664
Epoch 2, loss = 0.05237944424152374
Epoch 3, loss = 0.048090822994709015
Epoch 4, loss = 0.02320609986782074
Epoch 5, loss = 0.02974412962794304
Epoch 6, loss = 0.0327458418905735
Epoch 7, loss = 0.03187898173928261
Epoch 8, loss = 0.023978877812623978
Epoch 9, loss = 0.0343267060816288
Epoch 10, loss = 0.03831883892416954
Epsilon: 0.20782626385054587, Delta: 1e-05


C:\Users\Administrator\anaconda3\envs\torch\lib\site-packages\opacus\accountants\analysis\rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


Loss: 0.0001930263638496399, Accuracy: 0.2496


In [26]:
# Learning rate for training
learning_rate = 0.5
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 2.
# Clipping norm
clip_bound = 1.5
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.005282752215862274
Epoch 2, loss = 0.006581668742001057
Epoch 3, loss = 0.0058989133685827255
Epoch 4, loss = 0.007384671363979578
Epoch 5, loss = 0.010155069641768932
Epoch 6, loss = 0.016776299104094505
Epoch 7, loss = 0.01580929011106491
Epoch 8, loss = 0.013244871981441975
Epoch 9, loss = 0.014813763089478016
Epoch 10, loss = 0.016258051618933678
Epsilon: 0.6054551619620246, Delta: 1e-05
Loss: 0.0001400646924972534, Accuracy: 0.8585


In [28]:
# Learning rate for training
learning_rate = 0.3
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 3.
# Clipping norm
clip_bound = 1.
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 20
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.005998621694743633
Epoch 2, loss = 0.006012042053043842
Epoch 3, loss = 0.004418761469423771
Epoch 4, loss = 0.005905027035623789
Epoch 5, loss = 0.006454506888985634
Epoch 6, loss = 0.0077430484816432
Epoch 7, loss = 0.006105558946728706
Epoch 8, loss = 0.005686644464731216
Epoch 9, loss = 0.00818079337477684
Epoch 10, loss = 0.011749284341931343
Epoch 11, loss = 0.01217640656977892
Epoch 12, loss = 0.011672627180814743
Epoch 13, loss = 0.01663767732679844
Epoch 14, loss = 0.01435843389481306
Epoch 15, loss = 0.012777172029018402
Epoch 16, loss = 0.013146179728209972
Epoch 17, loss = 0.013701868243515491
Epoch 18, loss = 0.01678811013698578
Epoch 19, loss = 0.01779244840145111
Epoch 20, loss = 0.017634298652410507
Epsilon: 0.5358015231964641, Delta: 1e-05
Loss: 0.00011764787435531616, Accuracy: 0.8553


In [29]:
# Learning rate for training
learning_rate = 0.2
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.6
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.00932625774294138
Epoch 2, loss = 0.0064944312907755375
Epoch 3, loss = 0.0036767662968486547
Epoch 4, loss = 0.004332357551902533
Epoch 5, loss = 0.002935884054750204
Epoch 6, loss = 0.005231067538261414
Epoch 7, loss = 0.0031102898065000772
Epoch 8, loss = 0.003300727577880025
Epoch 9, loss = 0.0051866318099200726
Epoch 10, loss = 0.00507062254473567
Epsilon: 0.8136647699145032, Delta: 1e-05
Loss: 3.0039425939321517e-06, Accuracy: 0.9188


In [30]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.009068525396287441
Epoch 2, loss = 0.007501861546188593
Epoch 3, loss = 0.0038892205338925123
Epoch 4, loss = 0.005079340189695358
Epoch 5, loss = 0.0033665813971310854
Epsilon: 1.4824070580033397, Delta: 1e-05
Loss: 1.533075124025345e-05, Accuracy: 0.8767


In [31]:
# Learning rate for training
learning_rate = 0.4
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.5
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.006191704887896776
Epoch 2, loss = 0.006120648700743914
Epoch 3, loss = 0.003174602286890149
Epoch 4, loss = 0.004008895251899958
Epoch 5, loss = 0.0032801807392388582
Epoch 6, loss = 0.005756950471550226
Epoch 7, loss = 0.0034871576353907585
Epoch 8, loss = 0.0033769498113542795
Epoch 9, loss = 0.004921657498925924
Epoch 10, loss = 0.004393521696329117
Epsilon: 0.8916069841308477, Delta: 1e-05
Loss: 3.9145979098975656e-08, Accuracy: 0.9331


In [32]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.004
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.003084031865000725
Epoch 2, loss = 0.0017990535125136375
Epoch 3, loss = 0.002012252574786544
Epoch 4, loss = 0.003299159463495016
Epoch 5, loss = 0.003086068667471409
Epsilon: 0.6188836247099821, Delta: 1e-05
Loss: 4.5920062810182574e-06, Accuracy: 0.9249


In [33]:
# Learning rate for training
learning_rate = 0.4
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.004
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.003079631133005023
Epoch 2, loss = 0.0013738359557464719
Epoch 3, loss = 0.0028438176959753036
Epoch 4, loss = 0.004077156540006399
Epoch 5, loss = 0.004997132811695337
Epsilon: 0.6188836247099821, Delta: 1e-05
Loss: 8.957058191299438e-07, Accuracy: 0.8984


In [34]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.008
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.007676322013139725
Epoch 2, loss = 0.006448304280638695
Epoch 3, loss = 0.0046762097626924515
Epoch 4, loss = 0.003859391203150153
Epoch 5, loss = 0.0033902975264936686
Epsilon: 0.8361046634978282, Delta: 1e-05
Loss: 3.926738351583481e-06, Accuracy: 0.878


In [35]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.002
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0019237279193475842
Epoch 2, loss = 0.002529165707528591
Epoch 3, loss = 0.002950016176328063
Epoch 4, loss = 0.008137031458318233
Epoch 5, loss = 0.00857428926974535
Epsilon: 0.48191032106995346, Delta: 1e-05
Loss: 0.0001335606575012207, Accuracy: 0.8042


In [36]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.003
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0014880017843097448
Epoch 2, loss = 0.0015044690808281302
Epoch 3, loss = 0.0012882747687399387
Epoch 4, loss = 0.00246347370557487
Epoch 5, loss = 0.0035814139991998672
Epsilon: 0.5539421596678942, Delta: 1e-05
Loss: 4.2136919498443604e-05, Accuracy: 0.8893


In [37]:
# Learning rate for training
learning_rate = 0.125
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.003
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0032873814925551414
Epoch 2, loss = 0.002078196732327342
Epoch 3, loss = 0.0013927429681643844
Epoch 4, loss = 0.0016172376926988363
Epoch 5, loss = 0.003160846885293722
Epoch 6, loss = 0.0018871542997658253
Epoch 7, loss = 0.001472540432587266
Epoch 8, loss = 0.001204586704261601
Epoch 9, loss = 0.001743166008964181
Epoch 10, loss = 0.000683935359120369
Epsilon: 0.6793064502497669, Delta: 1e-05
Loss: 1.5623224899172782e-06, Accuracy: 0.9169


In [38]:
# Learning rate for training
learning_rate = 0.05
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.003
# Number of epochs
num_epochs = 10
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.005910482723265886
Epoch 2, loss = 0.004320061299949884
Epoch 3, loss = 0.0026985779404640198
Epoch 4, loss = 0.0020844144746661186
Epoch 5, loss = 0.0029985825531184673
Epoch 6, loss = 0.0020999787375330925
Epoch 7, loss = 0.001447996823117137
Epoch 8, loss = 0.0012396490201354027
Epoch 9, loss = 0.0014371193246915936
Epoch 10, loss = 0.001125303446315229
Epsilon: 0.6793064502497669, Delta: 1e-05
Loss: 1.5159940719604492e-05, Accuracy: 0.8572


# Subsample train data

In [182]:
train_tensor, test_tensor = get_mnist_dataset(train_size=3000)

In [186]:
# Learning rate for training
learning_rate = 0.02
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 1.5
# Batch size as a fraction of full data size
sample_rate = 0.004
# Number of epochs
num_epochs = 20
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0072714658454060555
Epoch 2, loss = 0.005599441006779671
Epoch 3, loss = 0.004659594036638737
Epoch 4, loss = 0.0025474040303379297
Epoch 5, loss = 0.004874274600297213
Epoch 6, loss = 0.006577667314559221
Epoch 7, loss = 0.007730745244771242
Epoch 8, loss = 0.012412753887474537
Epoch 9, loss = 0.004126830026507378
Epoch 10, loss = 0.00927735585719347
Epoch 11, loss = 0.0016702455468475819
Epoch 12, loss = 0.01121425349265337
Epoch 13, loss = 0.00893873255699873
Epoch 14, loss = 0.004142715595662594
Epoch 15, loss = 0.018805280327796936
Epoch 16, loss = 0.01783066615462303
Epoch 17, loss = 0.01347443275153637
Epoch 18, loss = 0.015416118316352367
Epoch 19, loss = 0.021014664322137833
Epoch 20, loss = 0.011776248924434185
Epsilon: 1.0672803504742279, Delta: 1e-05
Loss: 0.00025326640605926513, Accuracy: 0.5656


### Take full dataset

In [187]:
train_tensor, test_tensor = get_mnist_dataset(train_size=60000)

In [188]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.003
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0014719408936798573
Epoch 2, loss = 0.0017010967712849379
Epoch 3, loss = 0.0018709610449150205
Epoch 4, loss = 0.0009801796404644847
Epoch 5, loss = 0.0011247111251577735
Epsilon: 0.5539421596678942, Delta: 1e-05
Loss: 8.326234295964241e-07, Accuracy: 0.9311


In [189]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.4
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0010992912575602531
Epoch 2, loss = 0.0010812802938744426
Epoch 3, loss = 0.000982529716566205
Epoch 4, loss = 0.0005915345391258597
Epoch 5, loss = 0.002678802004083991
Epsilon: 0.38015269074646874, Delta: 1e-05
Loss: 2.2683321731165052e-09, Accuracy: 0.9165


In [190]:
# Learning rate for training
learning_rate = 0.3
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.4
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 3
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0011292078997939825
Epoch 2, loss = 0.0009494057740084827
Epoch 3, loss = 0.0009113266132771969
Epsilon: 0.34999697730814633, Delta: 1e-05
Loss: 6.230343133211136e-08, Accuracy: 0.9064


In [191]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.5
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 2
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0010796352289617062
Epoch 2, loss = 0.0009286236600019038
Epsilon: 0.29078942973607796, Delta: 1e-05
Loss: 1.4755974523723125e-06, Accuracy: 0.8822


In [35]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.3
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.002
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0019237279193475842
Epoch 2, loss = 0.002529165707528591
Epoch 3, loss = 0.002950016176328063
Epoch 4, loss = 0.008137031458318233
Epoch 5, loss = 0.00857428926974535
Epsilon: 0.48191032106995346, Delta: 1e-05
Loss: 0.0001335606575012207, Accuracy: 0.8042


In [194]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.001
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

C:\Users\Administrator\anaconda3\envs\torch\lib\site-packages\opacus\privacy_engine.py:133: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
C:\Users\Administrator\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch 1, loss = 0.0006509917438961565
Epoch 2, loss = 0.0010080678621307015
Epoch 3, loss = 0.0015376530354842544
Epoch 4, loss = 0.0006745706195943058
Epoch 5, loss = 0.0005489647737704217
Epsilon: 0.7266407511899576, Delta: 1e-05
Loss: 1.3921691477298737e-05, Accuracy: 0.9079


In [195]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.5
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.001
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0011801021173596382
Epoch 2, loss = 0.00151817814912647
Epoch 3, loss = 0.003622174495831132
Epoch 4, loss = 0.003054977161809802
Epoch 5, loss = 0.0022164147812873125
Epsilon: 0.288758623479293, Delta: 1e-05
Loss: 0.00019924048185348511, Accuracy: 0.8136


In [196]:
# Learning rate for training
learning_rate = 0.25
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.2
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.001
# Number of epochs
num_epochs = 5
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0003449567884672433
Epoch 2, loss = 0.0012119680177420378
Epoch 3, loss = 0.002123486017808318
Epoch 4, loss = 0.0013365363702178001
Epoch 5, loss = 0.0005887767183594406
Epsilon: 0.47182569674103964, Delta: 1e-05
Loss: 9.161260724067688e-05, Accuracy: 0.8789


### Solutions focusing on accuracy rather than epsilon

In [197]:
# Learning rate for training
learning_rate = 0.1
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.
# Clipping norm
clip_bound = 0.5
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 20
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0010874260915443301
Epoch 2, loss = 0.001134716090746224
Epoch 3, loss = 0.001119724242016673
Epoch 4, loss = 0.0007937122718431056
Epoch 5, loss = 0.0014535702066496015
Epoch 6, loss = 0.000990800792351365
Epoch 7, loss = 0.001129625248722732
Epoch 8, loss = 0.0005048282910138369
Epoch 9, loss = 0.0006879009306430817
Epoch 10, loss = 0.0005330557469278574
Epoch 11, loss = 0.0002691051922738552
Epoch 12, loss = 0.0006228769198060036
Epoch 13, loss = 0.00038238742854446173
Epoch 14, loss = 0.00036568782525137067
Epoch 15, loss = 0.00024392975319642574
Epoch 16, loss = 0.0009237586054950953
Epoch 17, loss = 0.0010624260175973177
Epoch 18, loss = 0.00045225402573123574
Epoch 19, loss = 0.00042965877219103277
Epoch 20, loss = 0.00043918381561525166
Epsilon: 1.0649553579562177, Delta: 1e-05
Loss: 8.389782742597162e-08, Accuracy: 0.9465


In [198]:
# Learning rate for training
learning_rate = 0.1
# Ratio of the standard deviation to the clipping norm
noise_multiplier = 1.
# Clipping norm
clip_bound = 0.8
# Batch size as a fraction of full data size
sample_rate = 0.0015
# Number of epochs
num_epochs = 30
    
train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)

model = SampleConvNet()
epsilon = train(
    model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
    noise_multiplier, clip_bound, delta, verbose=True
)
print("Epsilon: {}, Delta: {}".format(epsilon, delta))
average_loss, total_accuracy = test(model, test_loader)
print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0010490822605788708
Epoch 2, loss = 0.000889830756932497
Epoch 3, loss = 0.0007502700318582356
Epoch 4, loss = 0.000569394207559526
Epoch 5, loss = 0.0013765111798420548
Epoch 6, loss = 0.0007615330978296697
Epoch 7, loss = 0.0005313268047757447
Epoch 8, loss = 0.0004097585624549538
Epoch 9, loss = 0.000529853452462703
Epoch 10, loss = 0.00012683945533353835
Epoch 11, loss = 0.00021222926443442702
Epoch 12, loss = 0.0007154532941058278
Epoch 13, loss = 0.00044153182534500957
Epoch 14, loss = 0.0005186291527934372
Epoch 15, loss = 0.0002556252002250403
Epoch 16, loss = 0.000923754763789475
Epoch 17, loss = 0.0014770939014852047
Epoch 18, loss = 0.0004877931496594101
Epoch 19, loss = 0.0005806299741379917
Epoch 20, loss = 0.0005085001466795802
Epoch 21, loss = 0.0012865723110735416
Epoch 22, loss = 0.002021565567702055
Epoch 23, loss = 0.00034536575549282134
Epoch 24, loss = 0.00017896539065986872
Epoch 25, loss = 0.0006112336413934827
Epoch 26, loss = 0.001503852545283

In [16]:
sample_rates = [0.001, 0.01, 0.03, 0.1]
num_epochs=1

for sample_rate in sample_rates:
    train_loader, test_loader = create_data_loaders(train_tensor, test_tensor, sample_rate)
    model = SampleConvNet()
    epsilon = train(
        model, train_loader, torch.optim.SGD, learning_rate, num_epochs,
        noise_multiplier, clip_bound, delta, verbose=True
    )
    print("Epsilon: {}, Delta: {}".format(epsilon, delta))
    average_loss, total_accuracy = test(model, test_loader)
    print("Loss: {}, Accuracy: {}".format(average_loss, total_accuracy))

Epoch 1, loss = 0.0033979681320488453
Epsilon: 0.35079168683259393, Delta: 1e-05
Loss: 0.00019720089435577392, Accuracy: 0.2055
Epoch 1, loss = 0.008230012841522694
Epsilon: 0.6415768935615331, Delta: 1e-05
Loss: 5.5314445495605466e-05, Accuracy: 0.7116
Epoch 1, loss = 0.04996268078684807
Epsilon: 1.0335671024371273, Delta: 1e-05
Loss: 0.00015408546924591064, Accuracy: 0.5883
Epoch 1, loss = 0.22305183112621307
Epsilon: 2.038718812572111, Delta: 1e-05
Loss: 0.00021851372718811036, Accuracy: 0.2797


In [12]:
# sample_rate=0.01 looks promising, let's zoom in and look around that
sample_rates = [0.006, 0.007, 0.008, 0.009, 0.012, 0.015]
for sample_rate in sample_rates:
    eps, _, _, acc = pipeline(
    learning_rate, noise_multiplier, clip_bound, 
    sample_rate, num_epochs, data
    )
    print("Epsilon: ", eps)

Loss: 8.185320347547531e-06, Accuracy: 0.9216
Loss: 1.2426486937329174e-07, Accuracy: 0.9194
Loss: 2.4844862520694734e-06, Accuracy: 0.9349
Loss: 1.8928551580756903e-07, Accuracy: 0.9318
Loss: 2.2502657026052476e-06, Accuracy: 0.9118
Loss: 1.7072588205337524e-05, Accuracy: 0.8833


In [20]:
sample_rate = 0.008

noise_multipliers = [0.5, 1., 1.3, 2]
for noise_multiplier in noise_multipliers:
    eps, _, _, acc = pipeline(
    learning_rate, noise_multiplier, clip_bound, 
    sample_rate, num_epochs, data, verbose=True
    )
    print("Epsilon: ", eps)

Loss: 8.183129248209297e-08, Accuracy: 0.9256
Epsilon:  11.209556906334038
Loss: 3.966782707720995e-07, Accuracy: 0.9338
Epsilon:  1.4824070580033397
Loss: 1.1773756705224513e-07, Accuracy: 0.9259
Epsilon:  0.8361046634978282
Loss: 1.3864339562132954e-07, Accuracy: 0.9116
Epsilon:  0.4222310450136726


In [19]:
# sample_rate=0.01 looks promising, let's zoom in and look around that
noise_multiplier = 1.
pipeline(
    learning_rate, noise_multiplier, clip_bound, 
    sample_rate, num_epochs, data, verbose=True
    )
print("Epsilon: ", eps)

Loss: 1.973552629351616e-07, Accuracy: 0.9285
Epsilon:  0.4222310450136726
